In [1]:
from math import ceil, sqrt

In [2]:
C = 10**12
S = 10**8
M = 10**5
F = 0 # TEMP

In [3]:
def negative_mass(inputs, outputs_num):
    """
    Calculates the negative component of the storage mass formula. Note that there is no dependency on output
    values but only on their count. The code is designed to maximize precision and to avoid intermediate overflows.  
    In practice, all calculations should be saturating, i.e., clamped between u64::MIN and u64::MAX  
    """
    inputs_num = len(inputs)
    if outputs_num == 1 or inputs_num == 1 or (outputs_num == 2 and inputs_num == 2):
        return sum(C//v for v in inputs)
    return inputs_num*(C//(sum(v for v in inputs)//inputs_num)) 

def storage_mass(inputs, outputs):
    N = negative_mass(inputs, outputs_num=len(outputs))
    P = sum(C//o for o in outputs)
    return max(P-N, 0)

def compute_mass(inputs, outputs):
    return 1000 * len(inputs) + 10 * len(outputs)

def mass(inputs, outputs):
    return max(storage_mass(inputs, outputs), compute_mass(inputs, outputs)) 

In [14]:
def build_txs(inputs, payment, verbose=False):
    txs = []
    while storage_mass(inputs, outputs=[payment, sum(inputs) - payment - F]) > M:
        T = sum(inputs) - F
        N = negative_mass(inputs, 2) 
        D = (M + N)*T/C
        alpha = (D - sqrt(D**2 - 4*D))/(2*D)              # single step optimization, taking the smaller solution 
        outputs = [ceil(alpha * T), T - ceil(alpha * T)]  # round *up* in order to not increase the mass above M
        txs.append((inputs, outputs))
        if verbose:
            print(storage_mass(inputs, outputs))
        inputs = outputs
    txs.append((inputs, [payment, sum(inputs) - payment - F]))
    if verbose:
        print(storage_mass(*txs[-1]))
    return txs

In [15]:
storage_mass([S], [S//2, S//2])

30000

In [16]:
txs = build_txs([S], int(S*0.009), verbose=True)
len(txs)

99999
99999
99999
99999
99999
99999
99998
99998
99998
99999
99998
11217


12